# Prepare the dataset

In [6]:
!nvidia-smi

Mon Jul 11 15:22:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A4000           Off  | 00000000:00:05.0 Off |                  Off |
| 41%   36C    P8    15W / 140W |      0MiB / 16117MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!pip install beir
!pip install tensorflow-text
!pip install farm-haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-hibhktdy
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-hibhktdy
  Resolved https://github.com/deepset-ai/haystack.git to commit 1706729e26a94b61c9c863a1997f8f6a9cb3f759
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval

from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch

import logging
import pathlib, os
import random

from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

Dataset

In [12]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
data_path = util.download_and_unzip(url, "datasets")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/5183 [00:00<?, ?it/s]

In [13]:
import pandas as pd
pd_corpus = pd.DataFrame(corpus)

Logging

In [14]:
pd_corpus.head(2)

,4983,5836,7912,18670,19238,33370,36474,54440,70115,70490,...,167944455,168265642,195317463,195680777,195683603,195689316,195689757,196664003,198133135,198309074
text,Alterations of the architecture of cerebral white matter in the developing h...,Myelodysplastic syndromes (MDS) are age-dependent stem cell malignancies tha...,ID elements are short interspersed elements (SINEs) found in high copy numbe...,DNA methylation plays an important role in biological processes in human hea...,Two human Golli (for gene expressed in the oligodendrocyte lineage)-MBP (for...,Glioblastomas are deadly cancers that display a functional cellular hierarch...,Realizing the full potential of human embryonic stem cells (hESCs) and induc...,BACKGROUND Microarrays offer great potential as a platform for molecular dia...,Summary. We consider the problem of comparing complex hierarchical models in...,"Likelihood ratios are one of the best measures of diagnostic accuracy, altho...",...,Anabolic steroids have been widely used in the beef cattle industry for over...,"With the current and predicted increase in world population, growing global ...",Inadequate initial treatment and delayed hemodynamic stabilization (HDS) may...,BACKGROUND Moderate differences in efficacy between adjuvant chemotherapy re...,"Neutrophils are the main effector cells during inflammation, but they can al...",BACKGROUND The main associations of body-mass index (BMI) with overall and c...,A key aberrant biological difference between tumor cells and normal differen...,A signaling pathway transmits information from an upstream system to downstr...,AIMS Trabecular bone score (TBS) is a surrogate indicator of bone microarchi...,Introduction: Among the inflammatory mediators involved in the pathogenesis ...
title,Microstructural development of human newborn cerebral white matter assessed ...,Induction of myelodysplasia by myeloid-derived suppressor cells.,"BC1 RNA, the transcript from a master gene for ID element amplification, is ...",The DNA Methylome of Human Peripheral Blood Mononuclear Cells,The human myelin basic protein gene is included within a 179-kilobase transc...,Targeting A20 Decreases Glioma Stem Cell Survival and Tumor Growth,Efficient targeting of expressed and silent genes in human ESCs and iPSCs us...,Empirical Bayesian models for analysing molecular serotyping microarrays,Bayesian measures of model complexity and fit,Simplifying likelihood ratios,...,Application of growth technologies in enhancing food security and sustainabi...,Metabolic modifiers as performance-enhancing technologies for livestock prod...,Assessment of hemodynamic efficacy and safety of 6% hydroxyethylstarch 130/0...,Comparisons between different polychemotherapy regimens for early breast can...,Invariant NKT cells modulate the suppressive activity of IL-10-secreting neu...,Body-mass index and cause-specific mortality in 900 000 adults: collaborativ...,Targeting metabolic remodeling in glioblastoma multiforme.,Signaling architectures that transmit unidirectional information despite ret...,"Association between pre-diabetes, type 2 diabetes and trabecular bone score:...","Adhesion molecules and chemokines: relation to anthropometric, body composit..."


In [15]:
pd_corpus.shape

(2, 5183)

In [16]:
for title, text in pd_corpus.iteritems():
  print(title)
  print()
  print()

4983


5836


7912


18670


19238


33370


36474


54440


70115


70490


72159


79447


87758


92308


92499


97884


102662


103007


104130


106301


116792


118568


120626


123859


140874


143251


152245


153744


159469


164189


164985


169264


175735


188911


195352


202259


207972


213017


219475


226488


236204


238409


243694


253672


263364


266641


275294


279052


285794


293661


301838


301866


306006


306311


308862


313394


313403


317204


323030


323335


327319


335029


341324


343052


344240


350542


356218


364522


365896


368506


371289


374902


380526


381602


393001


406733


409280


410286


418246


427082


427865


432261


435529


437924


439670


456304


457630


461550


463309


463533


464511


469066


470625


471735


471921


474325


485020


493346


496873


502591


502797


503050


515489


516867


519974


520579


544971


581832


583260


596817


597790


599582


600437


60

In [17]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [21]:
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever
from haystack.utils import fetch_archive_from_http

documents: List[Document] = []
for title, text in pd_corpus.iteritems():
  documents.append(Document(content=text['text'], meta={"name": title or ""}))

# documents = []
# for key, value in pd_corpus.iteritems():
#   text_dic = {
#       "text" : value
#   }
#   documents.append(text_dic)

# from haystack.document_stores import FAISSDocumentStore
# document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

# document_store.write_documents(documents)


from haystack.document_stores import FAISSDocumentStore
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

In [22]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)


Writing Documents:   0%|          | 0/5183 [00:00<?, ?it/s]

# Haystack Retriever

*   BaseGraphRetriever(BaseComponent) ！！！
*   BaseRetriever(BaseComponent)
*   BM25Retriever(BaseRetriever)
*   FilterRetriever(BM25Retriever)
*   TfidfRetriever(BaseRetriever)
*   DensePassageRetriever(BaseRetriever)
*   TableTextRetriever(BaseRetriever)
*   EmbeddingRetriever(BaseRetriever)
*   Text2SparqlRetriever(BaseGraphRetriever)！！！

See documentation at: https://github.com/deepset-ai/haystack/blob/master/docs/_src/api/api/retriever.md




In [23]:
# from haystack.retriever.dense import EmbeddingRetriever

# retriever = EmbeddingRetriever(
#    document_store=document_store,

#    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
#    model_format="sentence_transformers",
  
# )


# document_store.update_embeddings(retriever, batch_size=128)

from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)
# Important:
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation.
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once.
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever)


INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-ctx_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx_encoder-single-nq-base
INFO - haystack.document_stores.faiss -  Updating embeddings for 5183 docs...


Updating Embedding:   0%|          | 0/5183 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/5184 [00:00<?, ? Docs/s]

In [24]:
# reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

# from haystack.pipelines import ExtractiveQAPipeline
# pipe = ExtractiveQAPipeline(reader, retriever)
from haystack.pipelines import DocumentSearchPipeline
pipe = DocumentSearchPipeline(retriever)

In [25]:
from haystack.utils.export_utils import print_documents
prediction = pipe.run(
    query="	1 in 5 million in UK have abnormal PrP positivity.", params={"Retriever": {"top_k": 5}}
)
print_documents(prediction, max_text_len= 100, print_name = True, print_meta = True)




Query: 	1 in 5 million in UK have abnormal PrP positivity.

{   'content': 'BACKGROUND There is overwhelming evidence that behavioural '
               'factors influence health, but their combi...',
    'meta': {'name': '12794099', 'vector_id': '2769'},
    'name': '12794099'}

{   'content': 'BACKGROUND Control of blood pressure (BP) following renal '
               'transplantation may improve allograft and ...',
    'meta': {'name': '21616324', 'vector_id': '4093'},
    'name': '21616324'}

{   'content': 'OBJECTIVE To assess the robustness of patient responses to a '
               'new national survey of patient experien...',
    'meta': {'name': '841371', 'vector_id': '3792'},
    'name': '841371'}

{   'content': 'Diindolylmethane (DIM), a bioactive metabolite of '
               'indole-3-carbinol found in cruciferous vegetables,...',
    'meta': {'name': '20454006', 'vector_id': '4981'},
    'name': '20454006'}

{   'content': 'BACKGROUND Lowering the blood cholesterol level

# Evaluation

In [26]:
# If Docker is available: Start Elasticsearch as docker container
# from haystack.utils import launch_es
# launch_es()

# Alternative in Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [27]:
from haystack.utils import fetch_archive_from_http

# Download evaluation data, which is a subset of Natural Questions development set containing 50 documents with one question per document and multiple annotated answers
doc_dir = "data/tutorial5"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/nq_dev_subset_v2.json.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/nq_dev_subset_v2.json.zip to `data/tutorial5`


True

In [28]:
# make sure these indices do not collide with existing ones, the indices will be wiped clean before data is inserted
doc_index = "tutorial5_docs"
label_index = "tutorial5_labels"

In [29]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(
    host="localhost",
    username="",
    password="",
    index=doc_index,
    label_index=label_index,
    embedding_field="emb",
    embedding_dim=768,
    excluded_meta_data=["emb"],
)

In [30]:
from haystack.nodes import PreProcessor

# Add evaluation data to Elasticsearch Document Store
# We first delete the custom tutorial indices to not have duplicate elements
# and also split our documents into shorter passages using the PreProcessor
preprocessor = PreProcessor(
    split_by="word",
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)
document_store.delete_documents(index=doc_index)
document_store.delete_documents(index=label_index)

# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.
document_store.add_eval_data(
    filename="data/tutorial5/nq_dev_subset_v2.json",
    doc_index=doc_index,
    label_index=label_index,
    preprocessor=preprocessor,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [31]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  ===

In [32]:
# Initialize Reader
from haystack.nodes import FARMReader

reader = FARMReader("deepset/roberta-base-squad2", top_k=4, return_no_answer=True)

# Define a pipeline consisting of the initialized retriever and reader
from haystack.pipelines import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

INFO - haystack.modeling.infer -  Got ya 7 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \   /'\   /'\   / \   / \   /'\ 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
from haystack.schema import EvaluationResult, MultiLabel

# We can load evaluation labels from the document store
# We are also opting to filter out no_answer samples
document_store.update_embeddings(retriever=retriever)
eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)
eval_result = pipeline.eval(labels=eval_labels, params={"Retriever": {"top_k": 5}})

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 1334 docs ...


Updating embeddings:   0%|          | 0/1334 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/1344 [00:00<?, ? Docs/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 91.96 Batches/s]


In [36]:
retriever_result = eval_result["Retriever"]
retriever_result.head()

,multilabel_id,query,filters,context,gold_contexts,gold_id_match,context_match,answer_match,gold_id_or_answer_match,gold_id_and_answer_match,...,rank,document_id,gold_document_ids,gold_documents_id_match,gold_contexts_similarity,gold_answers_match,type,node,eval_mode,index
0,2689774299151743127,who is written in the book of life,b'null',(NGR) The Headies (NGR) Americas Dove Awards (US) Grammy Awards (US) Billboa...,"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,0.0,0.0,...,1.0,dbbb1512eed992a600dc9673fd9f524e-9,"[de2fd2f109e11213af1ea189fd1488a3-0, de2fd2f109e11213af1ea189fd1488a3-0]","[0.0, 0.0]","[38.29416884247171, 38.29416884247171]","[0.0, 0.0]",document,Retriever,integrated,0
1,2689774299151743127,who is written in the book of life,b'null',to Work for Diddy A Raisin in the Sun Diddy -- Dirty Money Fox programming (...,"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,0.0,0.0,...,2.0,9a122d19eb40097b1bf5d94c589e6f1e-9,"[de2fd2f109e11213af1ea189fd1488a3-0, de2fd2f109e11213af1ea189fd1488a3-0]","[0.0, 0.0]","[37.423846823324624, 37.423846823324624]","[0.0, 0.0]",document,Retriever,integrated,1
2,2689774299151743127,who is written in the book of life,b'null',(2000) Mary-Louise Parker / Gary Sinise (2001) Liam Neeson (2002) Harvey Fie...,"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,0.0,0.0,...,3.0,7eb12a23e4d5dea7b6a60eddef133f48-17,"[de2fd2f109e11213af1ea189fd1488a3-0, de2fd2f109e11213af1ea189fd1488a3-0]","[0.0, 0.0]","[35.94429939077458, 35.94429939077458]","[0.0, 0.0]",document,Retriever,integrated,2
3,2689774299151743127,who is written in the book of life,b'null',(2013) Edward Norton (2014) Sylvester Stallone (2015) Jeff Bridges (2016) Wi...,"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,0.0,0.0,...,4.0,9c0ddb7d3cc6f716e3c5035bc018564a-20,"[de2fd2f109e11213af1ea189fd1488a3-0, de2fd2f109e11213af1ea189fd1488a3-0]","[0.0, 0.0]","[34.551784160139256, 34.551784160139256]","[0.0, 0.0]",document,Retriever,integrated,3
4,2689774299151743127,who is written in the book of life,b'null',(NSW) Lucy Wicks LIB 51.14 Chisholm (Vic) Julia Banks LIB 51.24 Dunkley (Vic...,"[Book of Life - wikipedia Book of Life Jump to: navigation, search This arti...",0.0,0.0,0.0,0.0,0.0,...,5.0,bb4fe9cda290c6d14ebd18e2cc84e942-6,"[de2fd2f109e11213af1ea189fd1488a3-0, de2fd2f109e11213af1ea189fd1488a3-0]","[0.0, 0.0]","[36.60130718954249, 36.60130718954249]","[0.0, 0.0]",document,Retriever,integrated,4


In [37]:
reader_result = eval_result["Reader"]
reader_result.head()

,multilabel_id,query,filters,gold_answers,answer,context,exact_match,f1,exact_match_context_scope,f1_context_scope,...,offsets_in_document,gold_offsets_in_documents,gold_answers_exact_match,gold_answers_f1,gold_documents_id_match,gold_contexts_similarity,type,node,eval_mode,index
0,2689774299151743127,who is written in the book of life,b'null',"[all people considered righteous before God, every person who is destined fo...",,None,0.0,0.0,0.0,0.0,...,"[{'start': 0, 'end': 0}]","[{'start': 1107, 'end': 1149}, {'start': 374, 'end': 434}]","[0, 0]","[0, 0]","[0.0, 0.0]","[0.0, 0.0]",answer,Reader,integrated,0
1,2689774299151743127,who is written in the book of life,b'null',"[all people considered righteous before God, every person who is destined fo...",Bill Paxton,(1982) John Lithgow (1983) Tracey Walter (1984) Roddy McDowall (1985) Bill P...,0.0,0.0,0.0,0.0,...,"[{'start': 906, 'end': 917}]","[{'start': 1107, 'end': 1149}, {'start': 374, 'end': 434}]","[0, 0]","[0, 0]","[0.0, 0.0]","[34.0, 34.0]",answer,Reader,integrated,1
2,2689774299151743127,who is written in the book of life,b'null',"[all people considered righteous before God, every person who is destined fo...",Brendan Gleeson,(2005) Bill Nighy (2006) David Oyelowo (2007) Paul Giamatti (2008) Brendan ...,0.0,0.0,0.0,0.0,...,"[{'start': 431, 'end': 446}]","[{'start': 1107, 'end': 1149}, {'start': 374, 'end': 434}]","[0, 0]","[0, 0]","[0.0, 0.0]","[34.89932885906041, 34.89932885906041]",answer,Reader,integrated,2
3,2689774299151743127,who is written in the book of life,b'null',"[all people considered righteous before God, every person who is destined fo...",Diddy,to Work for Diddy A Raisin in the Sun Diddy -- Dirty Money Fox programming (...,0.0,0.0,0.0,0.0,...,"[{'start': 12, 'end': 17}]","[{'start': 1107, 'end': 1149}, {'start': 374, 'end': 434}]","[0, 0]","[0, 0]","[0.0, 0.0]","[40.0, 40.0]",answer,Reader,integrated,3
0,5532137288444069466,who was the girl in the video brenda got a baby,b'null',[Ethel ``Edy'' Proctor],,None,0.0,0.0,0.0,0.0,...,"[{'start': 0, 'end': 0}]","[{'start': 181, 'end': 202}]",[0],[0],[0.0],[0.0],answer,Reader,integrated,0


In [38]:
eval_result.save("../")

INFO - haystack.schema -  Saving evaluation results to ..


In [39]:
saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

Retriever - Recall (single relevant document): 0.04
Retriever - Recall (multiple relevant documents): 0.03
Retriever - Mean Reciprocal Rank: 0.02
Retriever - Precision: 0.024
Retriever - Mean Average Precision: 0.019166666666666665
Reader - F1-Score: 0.0044444444444444444
Reader - Exact Match: 0.0


In [40]:
pipeline.print_eval_report(saved_eval_result)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit:  0.04
                        |
                      Reader
                        |
                        | exact_match:   0.0
                        | exact_match_top_1:   0.0
                        | f1: 0.00444
                        | f1_top_1:   0.0
                        |
                      Output

                Wrong Reader Examples
Query: 
 	who is written in the book of life
Gold Answers: 
 	all people considered righteous before God
 	every person who is destined for Heaven or the World to Come
Gold Document Ids: 
 	de2fd2f109e11213af1ea189fd1488a3-0
 	de2fd2f109e11213af1ea189fd1488a3-0
Metrics: 
 	exact_match: 0.0
 	f1: 0.0
Answers: 
 	multilabel_id: 2689774299151743127
 	filters: b'null'


In [47]:
from haystack.pipelines import DocumentSearchPipeline, Pipeline
from haystack.nodes import TextConverter, BM25Retriever
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.retriever.dense import EmbeddingRetriever

text_converter = TextConverter()
# document_store = ElasticsearchDocumentStore(search_fields=["content", "name"], index="scifact_beir")
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

#retriever = BM25Retriever(document_store=document_store, top_k=1000)
retriever = EmbeddingRetriever(
   document_store=document_store,

   embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
   model_format="sentence_transformers",
  
)

# retriever = DensePassageRetriever(
#     document_store=document_store,
#     query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#     passage_embedding_model="facebook/dpr-question_encoder-single-nq-base",
#     max_seq_len_query=64,
#     max_seq_len_passage=256,
#     batch_size=16,
#     use_gpu=True,
#     embed_title=True,
#     use_fast_tokenizers=True,
#     top_k = 1000,
# )
index_pipeline = Pipeline()
index_pipeline.add_node(text_converter, name="TextConverter", inputs=["File"])
index_pipeline.add_node(retriever, name="Embedder", inputs=["TextConverter"])
index_pipeline.add_node(document_store, name="DocumentStore", inputs=["Embedder"])
# document_store.update_embeddings(retriever)

query_pipeline = DocumentSearchPipeline(retriever=retriever)

ndcg, _map, recall, precision = Pipeline.eval_beir(
    index_pipeline=index_pipeline, query_pipeline=query_pipeline, dataset="scifact"
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  ===

  0%|          | 0/5183 [00:00<?, ?it/s]

INFO - haystack.pipelines.base -  indexing 5183 documents...


Create embeddings:   0%|          | 0/5184 [00:00<?, ? Docs/s]

INFO - haystack.pipelines.base -  indexing finished.
100%|██████████| 300/300 [02:05<00:00,  2.38it/s]
INFO - haystack.pipelines.base -  Cleaning up: deleting index 'scifact_beir'...
WARNING - haystack.document_stores.elasticsearch -  Deletion of default index 'scifact_beir' detected. If you plan to use this index again, please reinstantiate 'ElasticsearchDocumentStore' in order to avoid side-effects.
INFO - haystack.document_stores.elasticsearch -  Index 'scifact_beir' deleted.
INFO - haystack.pipelines.base -  Retriever evaluation for k in: [1, 3, 5, 10, 100, 1000]


In [48]:
ndcg, _map, recall, precision

({'NDCG@1': 0.0,
  'NDCG@3': 0.0,
  'NDCG@5': 0.0,
  'NDCG@10': 0.0,
  'NDCG@100': 0.00108,
  'NDCG@1000': 0.02305},
 {'MAP@1': 0.0,
  'MAP@3': 0.0,
  'MAP@5': 0.0,
  'MAP@10': 0.0,
  'MAP@100': 0.0001,
  'MAP@1000': 0.00055},
 {'Recall@1': 0.0,
  'Recall@3': 0.0,
  'Recall@5': 0.0,
  'Recall@10': 0.0,
  'Recall@100': 0.00667,
  'Recall@1000': 0.19833},
 {'P@1': 0.0,
  'P@3': 0.0,
  'P@5': 0.0,
  'P@10': 0.0,
  'P@100': 7e-05,
  'P@1000': 0.00022})